# Analysing GITT data

PyProBE includes built-in analysis methods for pulsing experiments, which this example
will demonstrate.

First import the required libraries and data:

In [ ]:
%%capture
%pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

import pyprobe

%matplotlib inline

In [ ]:
info_dictionary = {
    "Name": "Sample cell",
    "Chemistry": "NMC622",
    "Nominal Capacity [Ah]": 0.04,
    "Cycler number": 1,
    "Channel number": 1,
}
data_directory = "../../../tests/sample_data/neware"

# Create a cell object
cell = pyprobe.Cell(info=info_dictionary)
cell.add_procedure(
    procedure_name="Sample",
    folder_path=data_directory,
    filename="sample_data_neware.parquet",
)
print(cell.procedure["Sample"].experiment_names)

We will plot the Break-in Cycles and Discharge Pulses:

In [ ]:
fig, ax = plt.subplots()
cell.procedure["Sample"].experiment("Break-in Cycles").plot(
    x="Time [hr]",
    y="Voltage [V]",
    ax=ax,
    label="Break-in Cycles",
    color="blue",
)
cell.procedure["Sample"].experiment("Discharge Pulses").plot(
    x="Time [hr]",
    y="Voltage [V]",
    ax=ax,
    label="Discharge Pulses",
    color="red",
)
ax.set_ylabel("Voltage [V]")

State-of-charge is a useful metric when working with battery data, however it must be carefully defined. PyProBE doesn't automatically calculate a value for cell SOC until instructed to by the user for this reason.

To add an SOC column to the data, we call `set_soc()` on the procedure. We are going to provide an argument to `reference_charge`. This will be the final charge of the break-in cycles. This argument instructs PyProBE that the final data point of this charge is our 100% SOC reference.

In [ ]:
reference_charge = cell.procedure["Sample"].experiment("Break-in Cycles").charge(-1)
cell.procedure["Sample"].set_soc(reference_charge=reference_charge)

fig, ax = plt.subplots()
cell.procedure["Sample"].experiment("Break-in Cycles").plot(
    x="Time [hr]",
    y="SOC",
    ax=ax,
    label="Break-in Cycles",
    color="blue",
)
cell.procedure["Sample"].experiment("Discharge Pulses").plot(
    x="Time [hr]",
    y="SOC",
    ax=ax,
    label="Discharge Pulses",
    color="red",
)
ax.set_ylabel("SOC")
plt.legend(loc="lower left")

Then we'll filter to only the pulsing experiment:

In [ ]:
pulsing_experiment = cell.procedure["Sample"].experiment("Discharge Pulses")

fig, ax = plt.subplots()
pulsing_experiment.plot(
    x="Experiment Time [hr]",
    y="Voltage [V]",
    ax=ax,
    label="Discharge Pulses",
    color="red",
)
ax.set_ylabel("Voltage [V]")
plt.legend(loc="lower left")

And then create our pulsing analysis object.

In [ ]:
from pyprobe.analysis import pulsing

pulse_object = pulsing.Pulsing(input_data=pulsing_experiment)

With the pulsing object we can separate out individual pulses:

In [ ]:
fig, ax = plt.subplots()
pulse_object.input_data.plot(
    x="Experiment Time [hr]",
    y="Voltage [V]",
    label="Full Experiment",
    color="blue",
    ax=ax,
)
pulse_object.pulse(4).plot(
    x="Experiment Time [hr]",
    y="Voltage [V]",
    label="Pulse 5",
    color="red",
    ax=ax,
)
ax.set_ylabel("Voltage [V]")

We can also extract key parameters from the pulsing experiment, with the `get_resistances()` method.

In [ ]:
pulse_resistances = pulsing.get_resistances(input_data=pulsing_experiment)
print(pulse_resistances.data)

The `get_resistances()` method can take an argument of a list of times at which to evaluate the resistance after the pulse, for instance at 10s after the pulse:

In [ ]:
pulse_resistances = pulsing.get_resistances(input_data=pulsing_experiment, r_times=[10])
print(pulse_resistances.data)

As a result object, the pulse summary can also be plotted:

In [ ]:
fig, ax = plt.subplots()
pulse_resistances.plot(x="SOC", y="R0 [Ohms]", ax=ax, label="R0", color="blue")
pulse_resistances.plot(x="SOC", y="R_10s [Ohms]", ax=ax, label="R_10s", color="red")
ax.set_ylabel("Resistance [Ohms]")